## spaCy matcher

Documentation: https://spacy.io/api/matcher

In [2]:
import spacy
from spacy.matcher import Matcher

In [3]:
nlp = spacy.load("en_core_web_sm")

## Adding pattern format

In [4]:
matcher = Matcher(nlp.vocab)
pattern = [{"LIKE_EMAIL": True}]
matcher.add("EMAIL_ADDRESS", [pattern])

In [5]:
doc = nlp("This is my email address: hcyjc10@nottingham.ac.uk")
matches = matcher(doc)

In [6]:
print(matches)

[(16571425990740197027, 6, 7)]


In [7]:
print(nlp.vocab[matches[0][0]].text)

EMAIL_ADDRESS


In [8]:
with open ("data/wiki_mlk.txt", "r") as f:
    text = f.read()

In [9]:
print(text)

Martin Luther King Jr. (born Michael King Jr.; January 15, 1929 â€“ April 4, 1968) was an American Baptist minister and activist who became the most visible spokesman and leader in the American civil rights movement from 1955 until his assassination in 1968. King advanced civil rights through nonviolence and civil disobedience, inspired by his Christian beliefs and the nonviolent activism of Mahatma Gandhi. He was the son of early civil rights activist and minister Martin Luther King Sr.

King participated in and led marches for blacks' right to vote, desegregation, labor rights, and other basic civil rights.[1] King led the 1955 Montgomery bus boycott and later became the first president of the Southern Christian Leadership Conference (SCLC). As president of the SCLC, he led the unsuccessful Albany Movement in Albany, Georgia, and helped organize some of the nonviolent 1963 protests in Birmingham, Alabama. King helped organize the 1963 March on Washington, where he delivered his famou

In [10]:
nlp = spacy.load("en_core_web_sm")

In [11]:
matcher = Matcher(nlp.vocab)
pattern = [{"POS": "PROPN"}] # POS = Part of Speech
matcher.add("PROPER_NOUN", [pattern])
doc = nlp(text)
matches = matcher(doc)
print(len(matches))

102


In [12]:
print(doc[1]) # doc is indexed like a list

Luther


In [13]:
for match in matches[:10]:
    print(match, doc[match[1]:match[2]])

(451313080118390996, 0, 1) Martin
(451313080118390996, 1, 2) Luther
(451313080118390996, 2, 3) King
(451313080118390996, 3, 4) Jr.
(451313080118390996, 6, 7) Michael
(451313080118390996, 7, 8) King
(451313080118390996, 8, 9) Jr.
(451313080118390996, 10, 11) January
(451313080118390996, 16, 17) April
(451313080118390996, 24, 25) Baptist


Proper nouns above are just individual tokens. We can make it multi-word tokens by adding attribute 'OP'

In [14]:
matcher = Matcher(nlp.vocab)
pattern = [{"POS": "PROPN", "OP": "+"}]
matcher.add("PROPER_NOUN", [pattern])
doc = nlp(text)
matches = matcher(doc)
print(len(matches))

175


In [15]:
for match in matches[:10]:
    print(match, doc[match[1]:match[2]])

(451313080118390996, 0, 1) Martin
(451313080118390996, 0, 2) Martin Luther
(451313080118390996, 1, 2) Luther
(451313080118390996, 0, 3) Martin Luther King
(451313080118390996, 1, 3) Luther King
(451313080118390996, 2, 3) King
(451313080118390996, 0, 4) Martin Luther King Jr.
(451313080118390996, 1, 4) Luther King Jr.
(451313080118390996, 2, 4) King Jr.
(451313080118390996, 3, 4) Jr.


They overlap each other, that's why we have to add greedy attribute

In [16]:
matcher = Matcher(nlp.vocab)
pattern = [{"POS": "PROPN", "OP": "+"}]
matcher.add("PROPER_NOUN", [pattern], greedy="LONGEST")
doc = nlp(text)
matches = matcher(doc)
print(len(matches))

61


In [17]:
for match in matches[:10]:
    print(match, doc[match[1]:match[2]])

(451313080118390996, 84, 89) Martin Luther King Sr.
(451313080118390996, 470, 475) Martin Luther King Jr. Day
(451313080118390996, 537, 542) Martin Luther King Jr. Memorial
(451313080118390996, 0, 4) Martin Luther King Jr.
(451313080118390996, 129, 133) Southern Christian Leadership Conference
(451313080118390996, 248, 252) Director J. Edgar Hoover
(451313080118390996, 6, 9) Michael King Jr.
(451313080118390996, 326, 329) Nobel Peace Prize
(451313080118390996, 423, 426) James Earl Ray
(451313080118390996, 464, 467) Congressional Gold Medal


But it's not sorted in sequential order, so we have to sort it first

In [18]:
matcher = Matcher(nlp.vocab)
pattern = [{"POS": "PROPN", "OP": "+"}]
matcher.add("PROPER_NOUN", [pattern], greedy="LONGEST")
doc = nlp(text)
matches = matcher(doc)
matches.sort(key = lambda x: x[1]) # sort by start index using lambda function
print(len(matches))

61


In [19]:
for match in matches[:10]:
    print(match, doc[match[1]:match[2]])

(451313080118390996, 0, 4) Martin Luther King Jr.
(451313080118390996, 6, 9) Michael King Jr.
(451313080118390996, 10, 11) January
(451313080118390996, 16, 17) April
(451313080118390996, 24, 25) Baptist
(451313080118390996, 50, 51) King
(451313080118390996, 70, 72) Mahatma Gandhi
(451313080118390996, 84, 89) Martin Luther King Sr.
(451313080118390996, 90, 91) King
(451313080118390996, 114, 115) King


Maybe that isn't enough, we want to grab multi-word PROPN tokens that are always followed by a verb

In [20]:
matcher = Matcher(nlp.vocab)
pattern = [{"POS": "PROPN", "OP": "+"}, {"POS": "VERB"}]
matcher.add("PROPER_NOUN", [pattern], greedy="LONGEST")
doc = nlp(text)
matches = matcher(doc)
matches.sort(key = lambda x: x[1]) # sort by start index using lambda function
print(len(matches))

7


In [21]:
for match in matches[:10]:
    print(match, doc[match[1]:match[2]])

(451313080118390996, 50, 52) King advanced
(451313080118390996, 90, 92) King participated
(451313080118390996, 114, 116) King led
(451313080118390996, 168, 170) King helped
(451313080118390996, 248, 253) Director J. Edgar Hoover considered
(451313080118390996, 323, 325) King won
(451313080118390996, 486, 489) United States beginning


## Applying spaCy matchers on another text

In [23]:
import json
with open ("data/alice.json", "r") as f:
    data = json.load(f)

In [24]:
text = data[0][2][0]
print(text)

Alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do: once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it, `and what is the use of a book,' thought Alice `without pictures or conversation?'


Standardise the quotation mark first

In [29]:
text = text.replace("`","'")
print(text)

Alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do: once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it, 'and what is the use of a book,' thought Alice 'without pictures or conversation?'


In [30]:
matcher = Matcher(nlp.vocab)
pattern = [{"ORTH": "'"}, # ORTH means exact text match
           {"IS_ALPHA": True, "OP": "+"}, # OP + means 1 or more times, IS_ALPHA means alphabetic characters
           {"IS_PUNCT": True, "OP": "*"}, # OP * means 0 or more times
           {"ORTH": "'"}
        ]
matcher.add("PROPER_NOUN", [pattern], greedy="LONGEST")
doc = nlp(text)
matches = matcher(doc)
matches.sort(key = lambda x: x[1]) # sort by start index using lambda function
print(len(matches))

2


In [31]:
for match in matches[:10]:
    print(match, doc[match[1]:match[2]])

(451313080118390996, 47, 58) 'and what is the use of a book,'
(451313080118390996, 60, 67) 'without pictures or conversation?'


Our task is not only to extract, we need to also identify who's the speaker

In [35]:
speak_lemmas = ["think", "say"]
matcher = Matcher(nlp.vocab)
pattern = [{"ORTH": "'"}, # ORTH means exact text match
           {"IS_ALPHA": True, "OP": "+"}, # OP + means 1 or more times, IS_ALPHA means alphabetic characters
           {"IS_PUNCT": True, "OP": "*"}, # OP * means 0 or more times
           {"ORTH": "'"},
           {"POS": "VERB", "LEMMA": {"IN": speak_lemmas}},
           {"POS": "PROPN", "OP": "+"}
        ]
matcher.add("PROPER_NOUN", [pattern], greedy="LONGEST")
doc = nlp(text)
matches = matcher(doc)
matches.sort(key = lambda x: x[1]) # sort by start index using lambda function
print(len(matches))

1


In [36]:
for match in matches[:10]:
    print(match, doc[match[1]:match[2]]) # match[1] is the start index, match[2] is the end index

(451313080118390996, 47, 60) 'and what is the use of a book,' thought Alice


Now we have successfully extracted texts enclosed in quotes and identified who the speaker is

In [38]:
for text in data[0][2]:
    text = text.replace("`","'")
    doc = nlp(text)
    matches = matcher(doc)
    matches.sort(key = lambda x: x[1])
    for match in matches:
        print(match, doc[match[1]:match[2]])

(451313080118390996, 47, 60) 'and what is the use of a book,' thought Alice
(451313080118390996, 0, 6) 'Well!' thought Alice
(451313080118390996, 57, 68) 'which certainly was not here before,' said Alice
